# Simulation of a single freely-jointed chain

In [1]:
import unittest
import gen_chain
import math
import feasst as fst
print(fst.version())

def end_to_end_distance(mc):
    config = mc.system().configuration()
    pos = config.particle(0).site(0).position()
    return pos.distance(config.particle(0).site(config.particle(0).num_sites() - 1).position())

def radius_of_gyration(mc):
    config = mc.system().configuration()
    r_cm = fst.Position(config.dimension())
    n = config.particle(0).num_sites()
    for i in range(n):
        r_cm.add(config.particle(0).site(i).position())
    r_cm.divide(n)
    rg = 0
    for i in range(n):
        rg += config.particle(0).site(i).position().squared_distance(r_cm)
    return rg/n
        
class TestFreelyJointedIdealChain(unittest.TestCase):
    def test(self):
        n=10000
        gen_chain.linear_single_site(n, "data.chain.swp")
        mc = fst.MonteCarlo()
        mc.set(fst.MakeRandomMT19937(fst.args({"seed": "time"})))
        mc.add(fst.Configuration(fst.MakeDomain(fst.args({"cubic_box_length": str(3*n)})),
                                 fst.args({"particle_type": "data.chain.swp"})))
        mc.add(fst.MakePotential(fst.DontVisitModel()))
        mc.set(fst.MakeThermoParams(fst.args({"beta": "1", "chemical_potential0": "1"})))
        mc.set(fst.MakeMetropolis())
        fst.SeekNumParticles(1).with_trial_add().run(mc)
        mc.add(fst.MakeTrialGrowLinear(fst.MakeTrialComputeMove(), fst.args({"particle_type": "0"})))
        steps_per = str(int(1))
        mc.add(fst.MakeLogAndMovie(fst.args({"steps_per": steps_per, "file_name": "chain"})))
        end_to_end = fst.Accumulator()
        rg2 = fst.Accumulator()
        for i in range(int(1e2)):
            mc.attempt()
            end_to_end.accumulate(end_to_end_distance(mc))
            rg2.accumulate(radius_of_gyration(mc))
        print(end_to_end.str())
        print(rg2.str())
        self.assertAlmostEqual(end_to_end.average(), math.sqrt(n), delta=20)
        self.assertAlmostEqual(rg2.average(), n/6, delta=200)

v0.16.0-30-gbf11101ab9-hwh/doc2


In [2]:
unittest.main(argv=[''], verbosity=2, exit=False)

test (__main__.TestFreelyJointedIdealChain) ... 

average,stdev,block_stdev,n,moment0,moment1,
90.471539372386275,37.931249801668031,0,100,9047.1539372386275,960949.13508106156,
average,stdev,block_stdev,n,moment0,moment1,
1568.7441287256804,788.25473239985126,0,100,156874.41287256804,307609020.93305477,


ok

----------------------------------------------------------------------
Ran 1 test in 18.710s

OK


Did this tutorial work as expected? Did you find any inconsistencies or have any comments? Please [contact](../../../CONTACT.rst) us. Any feedback is appreciated!